# Полиномиальная регрессия (Polynomial Regression)

In [18]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
# возьмем данные о домах в Калифорнии
from sklearn.datasets import fetch_california_housing

> **Полиномиальная регрессия (Polynomial Regression)** — это более сложная модель, чем линейная регрессия. \
Вместо уравнения прямой используется уравнение полинома (многочлена). \
Степень полинома может быть сколь угодно большой: чем больше степень, тем сложнее модель.

Например, если в исходных данных было два признака $x_1$ и $x_2$.\
То после генерации полиномиальных признаков мы получим 5 признаков:\
$ x_1, x_2, x_1^2, x_2^2, x_1x_2,$

Полиномиальные признаки можно создать с помощью объекта класса [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) 
из модуля `preprocessing`. \
Это преобразователь, который позволяет сгенерировать полиномиальные признаки любой степени \
и добавить их в таблицу. \
У него есть два важных параметра:

- `degree` — степень полинома. По умолчанию используется степень `2` \
(лучше не указывать больше `3`-х, так как возрастает риск переобучения).
- `include_bias` — включать ли в результирующую таблицу столбец из единиц (x в степени 0). \
По умолчанию стоит `True`, лучше выставить его в значение `False`, \
так как столбец из единиц и так добавляется в методе наименьших квадратов.

Получим данные о домах в калифорнии и подготовим их для передачи в модель.

In [19]:
data = fetch_california_housing(as_frame=True)

housing_data = data['frame']
housing_data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [20]:
# признаки
X = housing_data.drop(['MedHouseVal'], axis=1)
# целевой признак
y = housing_data['MedHouseVal']

# копируем названия столбцов, которые теряются при использовании fit_transform()
feature_names = list(X.columns)

# нормализуем данные, чтобы коэффициенты не были слишком большими, так модель будет устойчивой

# инициализируем нормализатор MinMaxScaler
mm_scaler = preprocessing.MinMaxScaler()

# производим стандартизацию
X_scaled = mm_scaler.fit_transform(X)

# составляем DataFrame из результата
X_scaled = pd.DataFrame(X_scaled, columns=feature_names)

X_scaled.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,0.539668,0.784314,0.043512,0.020469,0.008941,0.001499,0.567481,0.211155
1,0.538027,0.392157,0.038224,0.018929,0.067210,0.001141,0.565356,0.212151
2,0.466028,1.000000,0.052756,0.021940,0.013818,0.001698,0.564293,0.210159
3,0.354699,1.000000,0.035241,0.021929,0.015555,0.001493,0.564293,0.209163
4,0.230776,1.000000,0.038534,0.022166,0.015752,0.001198,0.564293,0.209163


In [21]:
# разделяем на тренировочную и тестовую выборки
X_train_scaled, X_test_scaled, y_train, y_test = \
    train_test_split(X_scaled, y, test_size=0.25, random_state=42)

In [22]:
# cоздаём генератор полиномиальных признаков
polynomial_features_generator = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
# обучаем его
polynomial_features_generator.fit(X_train_scaled)

# генерируем полиномиальные признаки для тренировочной выборки
X_train_poly = polynomial_features_generator.transform(X_train_scaled)

# генерируем полиномиальные признаки для тестовой выборки
X_test_poly = polynomial_features_generator.transform(X_test_scaled)

# посмотрим на количество созданных признаков
print(X_train_poly.shape[1])
print(X_test_poly.shape[1])

44
44


Тип возвращаемых данных - `numpy`-массив.

In [23]:
print(type(X_train_poly))
print(type(X_test_poly))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Передадим новые признаки в линейную регрессию.

In [24]:
# создаём объект класса LinearRegression
linear_regression_poly_model = linear_model.LinearRegression()

# обучаем модель
linear_regression_poly_model.fit(X_train_poly, y_train)

# делаем предсказание для тренировочной выборки
y_train_pred_poly = linear_regression_poly_model.predict(X_train_poly)

# делаем предсказание для тестовой выборки
y_test_pred_poly = linear_regression_poly_model.predict(X_test_poly)

# посчитаем MAPE для тренировочной выборки
mape_train = metrics.mean_absolute_percentage_error(y_train, y_train_pred_poly)
# посчитаем MAPE для тренировочной выборки
mape_test = metrics.mean_absolute_percentage_error(y_test, y_test_pred_poly)

print('MAPE train:', mape_train)
print('MAPE test:', mape_test)

MAPE train: 0.2616542118066109
MAPE test: 0.2699298457648717


Метрики примерно одинаковы, значит переобучения нет.\
Значение метрики лучше, чем для [обычной линейной регрессии](./linear_regression.ipynb) (0.31978371646123604)

In [25]:
# свободный член w0
print('w0: {}'.format(linear_regression_poly_model.intercept_)) 

# остальные параметры модели w1, w2, ..., wm
print('wi: {}'.format(linear_regression_poly_model.coef_))

w0: 3.2323200395524423
wi: [ 2.52900498e+01  3.51800485e+00 -1.52468107e+02  1.71331144e+02
 -1.56892678e+01 -3.58575739e+02 -7.06230962e+00 -5.99952572e+00
 -6.43248386e+00  1.18175064e+00  7.91427757e+01 -6.78566465e+01
  2.64578650e+01 -6.92815054e+01 -2.20679953e+01 -2.20883158e+01
  5.48888465e-01 -4.30271578e+00  1.87664532e+01  4.66868674e+00
 -1.24963165e+02 -4.96491100e+00 -5.15429266e+00  2.32548690e+02
 -5.54072764e+02 -2.77680806e+02  3.74402272e+03  1.42012315e+02
  1.42480181e+02  3.28648710e+02  5.88457782e+02 -3.62782319e+03
 -1.71426124e+02 -1.68050938e+02  1.80790851e+00  1.08219429e+03
  3.52683219e+00  1.30415893e+00  8.12143282e+01  2.38295375e+02
  1.98097051e+02  5.13997758e+00  9.60459908e+00  3.84716685e+00]


Коэффициенты не очень большие, можно предположить, что модель устойчива.